本节将介绍如何预处理一个语言模型数据集，并将其转换成**字符级循环神经网络**所需要的输入格式。为此，我们收集了周杰伦从第一张专辑《Jay》到第十张专辑《跨时代》中的歌词，并在后面几节里应用循环神经网络来训练一个语言模型。当模型训练好后，我们就可以用这个模型来**创作歌词**。

## 6.3.1 读取数据集

首先读取这个数据集，看看前40个字符是什么样的。

In [1]:
import tensorflow as tf
import random
import zipfile

with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

这个数据集有6万多个字符。为了打印方便，我们把换行符替换成空格，然后仅使用前1万个字符来训练模型。

In [9]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

## 6.3.2 建立字符索引

我们将每个字符映射成一个从0开始的连续整数，又称索引，来方便之后的数据处理。为了得到索引，我们**将数据集里所有不同字符取出来**，然后将其逐一映射到索引来构造词典。接着，打印`vocab_size`，即词典中不同字符的个数，又称词典大小。

In [10]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char,i) for i,char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

1027

之后，将训练数据集中每个字符转化为索引，并打印前20个字符及其对应的索引。

In [11]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))  # 根据索引找到，其原来对应的 字符串
print('indices:', sample) 

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [469, 829, 870, 591, 711, 669, 31, 469, 829, 318, 746, 296, 873, 820, 996, 572, 31, 469, 829, 318]


**很好的想法。**

我们将以上代码封装在`d2lzh_tensorflow2`包里的`load_data_jay_lyrics`函数中，以方便后面章节调用。调用该函数后会依次得到`corpus_indices`、`char_to_idx`、`idx_to_char`和`vocab_size`这4个变量。